# Laplacian Convolutional Representation (LCR)

In [1]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def laplacian(T, tau):
    ell = np.zeros(T)
    ell[0] = 2 * tau
    for k in range(tau):
        ell[k + 1] = -1
        ell[-k - 1] = -1
    return ell

def prox(z, w, lmbda, denominator):
    T = z.shape[0]
    temp = np.fft.fft2(lmbda * z - w) / denominator
    temp1 = 1 - T / (lmbda * np.abs(temp))
    temp1[temp1 <= 0] = 0
    return np.fft.ifft2(temp * temp1).real

def update_z(y_train, pos_train, x, w, lmbda, eta):
    z = x + w / lmbda
    z[pos_train] = (lmbda / (lmbda + eta) * z[pos_train] 
                    + eta / (lmbda + eta) * y_train)
    return z

def update_w(x, z, w, lmbda):
    return w + lmbda * (x - z)

def LCR(y_true, y, lmbda, gamma, tau, maxiter = 50):
    eta = 100 * lmbda
    N, T = y.shape
    pos_train = np.where(y != 0)
    y_train = y[pos_train]
    pos_test = np.where((y_true != 0) & (y == 0))
    y_test = y_true[pos_test]
    z = y.copy()
    w = y.copy()
    ell_s = np.zeros(N)
    ell_s[0] = 1
    ell_t = laplacian(T, tau)
    denominator = lmbda + gamma * np.fft.fft2(np.outer(ell_s, ell_t)) ** 2
    del y_true, y
    show_iter = 10
    for it in range(maxiter):
        x = prox(z, w, lmbda, denominator)
        z = update_z(y_train, pos_train, x, w, lmbda, eta)
        w = update_w(x, z, w, lmbda)
        if (it + 1) % show_iter == 0:
            print(it + 1)
            print(compute_mape(y_test, x[pos_test]))
            print(compute_rmse(y_test, x[pos_test]))
            print()
    return x

In [2]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.3
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)

import time
start = time.time()
N, T = sparse_mat.shape
lmbda = 1e-9 * N * T
gamma = 5 * lmbda
tau = 1
maxiter = 100
x = LCR(dense_mat, sparse_mat, lmbda, gamma, tau, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

10
0.060260624064500855
4.611633445101357

20
0.019065953246163798
1.7187468215058075

30
0.017240989370844582
1.614893481537942

40
0.016590651250732287
1.5805236165262828

50
0.016276039301530824
1.5642941516913245

60
0.016092651225494543
1.5549414739764096

70
0.015970776772878555
1.5488013706462824

80
0.01588403776397548
1.5444757771706277

90
0.01581846863834731
1.5412087527797633

100
0.01576714586308319
1.5386640622909022

Running time: 1749 seconds.


In [3]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.5
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)

import time
start = time.time()
N, T = sparse_mat.shape
lmbda = 1e-9 * N * T
gamma = 5 * lmbda
tau = 1
maxiter = 100
x = LCR(dense_mat, sparse_mat, lmbda, gamma, tau, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

10
0.22153155476942565
18.69022559079149

20
0.023910111238298202
2.0224484647647714

30
0.019855289519879086
1.8111444967023937

40
0.018933762189000926
1.769787823319514

50
0.018555789916083632
1.7541306109638066

60
0.0183614687100596
1.7465511866555943

70
0.018246312977041473
1.7422658903356758

80
0.018170056385422333
1.7395351341258036

90
0.018115342857423312
1.737636750677174

100
0.018074535774616783
1.7362227837400492

Running time: 1714 seconds.


In [4]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.7
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)

import time
start = time.time()
N, T = sparse_mat.shape
lmbda = 1e-9 * N * T
gamma = 5 * lmbda
tau = 1
maxiter = 100
x = LCR(dense_mat, sparse_mat, lmbda, gamma, tau, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

10
0.4615953359505017
36.60477966892723

20
0.04037640599488146
3.067023737842156

30
0.026313217189542916
2.2513328588872703

40
0.02405968177489366
2.149835645166562

50
0.023389434829006486
2.124202893925095

60
0.023133212743871114
2.116497452469566

70
0.023005796435268074
2.113389697197214

80
0.022935736096471603
2.11207583715308

90
0.02289268409413733
2.111445812508813

100
0.02286435753084884
2.1111209322060827

Running time: 1698 seconds.


In [5]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.9
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)

import time
start = time.time()
N, T = sparse_mat.shape
lmbda = 1e-9 * N * T
gamma = 5 * lmbda
tau = 1
maxiter = 100
x = LCR(dense_mat, sparse_mat, lmbda, gamma, tau, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

10
0.7465827710753415
53.89622112745139

20
0.18639131726418076
14.230221106926132

30
0.07822495667435395
5.333054431727051

40
0.045650938113076744
3.524945715553431

50
0.04121184013331336
3.306250408781554

60
0.03930379850495469
3.218989966757169

70
0.038545652282843186
3.184568794630996

80
0.038240222374946606
3.171041020389246

90
0.03811119417505822
3.1655902422742477

100
0.03805115494017022
3.163194155719253

Running time: 1685 seconds.


### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>